In [1]:
from sqlalchemy import create_engine, text
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import numpy as np

from datetime import date

pd.set_option('display.max_columns', None)

NOTES:
- How to handle values that contain <1 & > 99 (drop numerators?)
- Drop blank testing records
- Combine all Examiners into 1 column 
- Drop duplicate records 
- Drop unnecessary columns 

In [2]:
# Cleaned data in Excel 
# Cleaned missing examiner info
# Cleaned date of testing
df = pd.read_excel('../data/bdi3_all_evals_clean.xlsx')

In [ ]:
df.head(1)

### Add 'Age at Testing' column for Q4

In [3]:
# Calculate the age at testing
df['Age at Testing'] = ((df['Date of Testing'] - df['Date of Birth']).dt.days) / 365
df.head()

,Child ID,Gender,Date of Birth,Location - Sub Level 1,Program Label,Date of Testing,Examiner,Adaptive Sum of Scaled Scores,Adaptive Developmental Quotient,Adaptive Percentile Rank,Adaptive 95% Confidence Interval,Adaptive RDI,Adaptive Z-Score,Adaptive T-Score,Adaptive NCE,Social-Emotional Sum of Scaled Scores,Social-Emotional Developmental Quotient,Social-Emotional Percentile Rank,Social-Emotional 95% Confidence Interval,Social-Emotional RDI,Social-Emotional Z-Score,Social-Emotional T-Score,Social-Emotional NCE,Communication Sum of Scaled Scores,Communication Developmental Quotient,Communication Percentile Rank,Communication 95% Confidence Interval,Communication RDI,Communication Z-Score,Communication T-Score,Communication NCE,Motor Sum of Scaled Scores,Motor Developmental Quotient,Motor Percentile Rank,Motor 95% Confidence Interval,Motor RDI,Motor Z-Score,Motor T-Score,Motor NCE,Cognitive Sum of Scaled Scores,Cognitive Developmental Quotient,Cognitive Percentile Rank,Cognitive 95% Confidence Interval,Cognitive RDI,Cognitive Z-Score,Cognitive T-Score,Cognitive NCE,BDI-3 Total Sum of Scaled Scores,BDI-3 Total Developmental Quotient,BDI-3 Total Percentile Rank,BDI-3 Total 90% Confidence Interval,BDI-3 Total 95% Confidence Interval,BDI-3 Total Z-Score,BDI-3 Total T-Score,BDI-3 Total NCE,Adaptive-Self Care RS,Adaptive-Self Care SS,Adaptive-Self Care PR,Adaptive-Self Care AE,Adaptive-Self Care RDI,Adaptive-Self Care CSS,Adaptive-Self Care CSS 90%,Adaptive-Self Care Z-Score,Adaptive-Self Care T-Score,Adaptive-Self Care NCE,Adaptive-Personal Responsibility RS,Adaptive-Personal Responsibility SS,Adaptive-Personal Responsibility PR,Adaptive-Personal Responsibility AE,Adaptive-Personal Responsibility RDI,Adaptive-Personal Responsibility CSS,Adaptive-Personal Responsibility CSS 90%,Adaptive-Personal Responsibility Z-Score,Adaptive-Personal Responsibility T-Score,Adaptive-Personal Responsibility NCE,Social Emotional-Adult Interaction RS,Social Emotional-Adult Interaction SS,Social Emotional-Adult Interaction PR,Social Emotional-Adult Interaction AE,Social Emotional-Adult Interaction RDI,Social Emotional-Adult Interaction CSS,Social Emotional-Adult Interaction CSS 90%,Social Emotional-Adult Interaction Z-Score,Social Emotional-Adult Interaction T-Score,Social Emotional-Adult Interaction NCE,Social Emotional-Peer Interaction RS,Social Emotional-Peer Interaction SS,Social Emotional-Peer Interaction PR,Social Emotional-Peer Interaction AE,Social Emotional-Peer Interaction RDI,Social Emotional-Peer Interaction CSS,Social Emotional-Peer Interaction CSS 90%,Social Emotional-Peer Interaction Z-Score,Social Emotional-Peer Interaction T-Score,Social Emotional-Peer Interaction NCE,Social Emotional-Self Concept / Social Role RS,Social Emotional-Self Concept / Social Role SS,Social Emotional-Self Concept / Social Role PR,Social Emotional-Self Concept / Social Role AE,Social Emotional-Self Concept / Social Role RDI,Social Emotional-Self Concept / Social Role CSS,Social Emotional-Self Concept / Social Role CSS 90%,Social Emotional-Self Concept / Social Role Z-Score,Social Emotional-Self Concept / Social Role T-Score,Social Emotional-Self Concept / Social Role NCE,Communication-Receptive Communication RS,Communication-Receptive Communication SS,Communication-Receptive Communication PR,Communication-Receptive Communication AE,Communication-Receptive Communication RDI,Communication-Receptive Communication CSS,Communication-Receptive Communication CSS 90%,Communication-Receptive Communication Z-Score,Communication-Receptive Communication T-Score,Communication-Receptive Communication NCE,Communication-Expressive Communication RS,Communication-Expressive Communication SS,Communication-Expressive Communication PR,Communication-Expressive Communication AE,Communication-Expressive Communication RDI,Communication-Expressive Communication CSS,Communication-Expressive Communication CSS 90%,Communication-Expressive Communication Z-Score,Communication-Expressive Comm

### Clean RDI Column - Drop numerator & apply to all RDI columns

In [4]:
# Andri's function to clean RDI
def clean_rdi(column):
    months = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
    adaptive_rdi = df[column].str[0:-3]
    adaptive_rdi = [months[i] if i in months.keys() else i for i in adaptive_rdi] 
    adaptive_rdi = [0 if str(i)[0:1] == '<' else i for i in adaptive_rdi] 
    adaptive_rdi = [100 if str(i)[0:1] == '>' else i for i in adaptive_rdi] 
    adaptive_rdi = pd.to_numeric(adaptive_rdi)
    return adaptive_rdi

In [5]:
# Apply function to all columns with 'RDI'
for column in df.columns:
    if 'RDI' in column:
        df[column] = clean_rdi(column)

In [ ]:
df.sort_values('Gender')

### String Cleaning

In [6]:
# Swtich to strings
df[['Gender', 'Location - Sub Level 1', 'Program Label', 'Examiner']] = df[['Gender', 'Location - Sub Level 1', 'Program Label', 'Examiner']].astype('string')
#df.info()

### Clean 'interval' columns by converting to strings

In [7]:
# Clean String Function
def string_clean(column):
    x = df[column].astype('string')
    return x

In [8]:
# Apply to all confidence intervals 
for column in df.columns:
    if 'Interval' in column:
        df[column] = string_clean(column)

### Switch 'Score' columns to numeric

In [9]:
df.groupby('Motor Z-Score')['Motor Z-Score'].count()

Motor Z-Score
0         1440
0.13       651
0.2        741
0.27       663
0.4       1249
          ... 
– 3.27      74
– 3.40      94
– 3.60     127
– 3.73      18
– 3.87     119
Name: Motor Z-Score, Length: 65, dtype: int64

In [15]:
df['Motor Z-Score'] = df['Motor Z-Score'].astype(str).replace('– ', '-', regex=True)
df['Motor Z-Score'] = df['Motor Z-Score'].replace('--', 0, regex=True)
df['Motor Z-Score'] = df['Motor Z-Score'].apply(lambda x: float(x.replace('– ', '')) * -1 if '– ' in x else float(x))
df['Motor Z-Score'] = df['Motor Z-Score'].fillna(0)
#df.groupby('Motor Z-Score')['Motor Z-Score'].count()
#df['Motor Z-Score'] 

In [21]:
def clean_z(column):
    z_col = df['Motor Z-Score'].astype(str).replace('– ', '-', regex=True)
    z_col = z_col.replace('--', 0, regex=True)
    z_col = z_col.apply(lambda x: float(x.replace('– ', '')) * -1 if '– ' in x else float(x))
    z_col = z_col.fillna(0)
    return z_col

In [22]:
for column in df.columns:
    if 'Z-Score' in column:
        df[column] = clean_z(column)

In [24]:
z_scores = [col for col in df.columns if 'Z-Score' in col]
z = df[z_scores]
z.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23944 entries, 0 to 23943
Data columns (total 19 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Adaptive Z-Score                                     23944 non-null  float64
 1   Social-Emotional Z-Score                             23944 non-null  float64
 2   Communication Z-Score                                23944 non-null  float64
 3   Motor Z-Score                                        23944 non-null  float64
 4   Cognitive Z-Score                                    23944 non-null  float64
 5   BDI-3 Total Z-Score                                  23944 non-null  float64
 6   Adaptive-Self Care Z-Score                           23944 non-null  float64
 7   Adaptive-Personal Responsibility Z-Score             23944 non-null  float64
 8   Social Emotional-Adult Interaction Z-Score           23944 non-nul

In [26]:
z.head()

,Adaptive Z-Score,Social-Emotional Z-Score,Communication Z-Score,Motor Z-Score,Cognitive Z-Score,BDI-3 Total Z-Score,Adaptive-Self Care Z-Score,Adaptive-Personal Responsibility Z-Score,Social Emotional-Adult Interaction Z-Score,Social Emotional-Peer Interaction Z-Score,Social Emotional-Self Concept / Social Role Z-Score,Communication-Receptive Communication Z-Score,Communication-Expressive Communication Z-Score,Motor-Gross Motor Z-Score,Motor-Fine Motor Z-Score,Motor-Perceptual Motor Z-Score,Cognitive-Attention and Memory Z-Score,Cognitive-Reasoning / Academic Skills Z-Score,Cognitive-Perception and Concepts Z-Score
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,-3.87,-3.87,-3.87,-3.87,-3.87,-3.87,-3.87,-3.87,-3.87,-3.87,-3.87,-3.87,-3.87,-3.87,-3.87,-3.87,-3.87,-3.87,-3.87
3,-3.87,-3.87,-3.87,-3.87,-3.87,-3.87,-3.87,-3.87,-3.87,-3.87,-3.87,-3.87,-3.87,-3.87,-3.87,-3.87,-3.87,-3.87,-3.87
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [41]:
# Using Amy's function to create column with eligibility
def filter(x):
    if x <= 70:
        return '40% delay'
    if (x >= 71 and x <= 78):
        return '25% delay'
    if x >= 79:
        return 'not eligible'
#applying the filter function to 'Salary' column 
df['Eligibility'] = df['BDI-3 Total Developmental Quotient'].apply(filter)

# MVP

## 4. Does the child's age seem to impact their scoring?

In [37]:
general = df.iloc[:, :7]

,Child ID,Gender,Date of Birth,Location - Sub Level 1,Program Label,Date of Testing,Examiner
0,404042,M,2020-03-17,Upper Cumberland,BDI-3 Annual Evaluation,2023-03-09,Stacy Holden
1,404042,M,2020-03-17,Upper Cumberland,BDI-3 Annual Evaluation,2023-03-09,Stacy Holden
2,475636,F,2019-10-16,Northwest,BDI-3 Annual Evaluation,2023-07-19,Jennifer Hodges
3,475636,F,2019-10-16,Northwest,BDI-3 Annual Evaluation,2023-07-19,Jennifer Hodges
4,476193,M,2019-11-08,Memphis Delta,BDI-3 Annual Evaluation,2023-02-27,Kayla Coley
...,...,...,...,...,...,...,...
23939,544499,M,2020-11-25,Greater Nashville,BDI-3 Eligibility Evaluation,2023-01-17,Tracey Hardiman
23940,549646,M,2020-05-02,Southeast Tennessee,BDI-3 Eligibility Evaluation,2023-08-21,Mollee Favre
23941,552599,F,2022-08-27,First Tennessee,BDI-3 Eligibility Evaluation,2023-05-09,Karen McMillion
23942,SP00659,M,2020-07-30,East Tennessee,BDI-3 Eligibility Evaluation,2023-05-22,Kelly Brackney


In [38]:
bdi_totals = [col for col in df.columns if 'BDI-3 Total' in col]
bdi = df[bdi_totals]
bdi

,BDI-3 Total Sum of Scaled Scores,BDI-3 Total Developmental Quotient,BDI-3 Total Percentile Rank,BDI-3 Total 90% Confidence Interval,BDI-3 Total 95% Confidence Interval,BDI-3 Total Z-Score,BDI-3 Total T-Score,BDI-3 Total NCE
0,15.0,40.0,<0.1,39-43,38-44,0.00,NaN,NaN
1,15.0,40.0,<0.1,39-43,38-44,0.00,NaN,NaN
2,24.0,44.0,<0.1,43-47,42-48,-3.87,13.0,<1
3,24.0,44.0,<0.1,43-47,42-48,-3.87,13.0,<1
4,37.0,51.0,<0.1,49-53,48-54,0.00,17.0,<1
...,...,...,...,...,...,...,...,...
23939,NaN,NaN,NaN,<NA>,<NA>,0.00,NaN,NaN
23940,NaN,NaN,NaN,<NA>,<NA>,-3.60,NaN,NaN
23941,NaN,NaN,NaN,<NA>,<NA>,-0.40,NaN,NaN
23942,NaN,NaN,NaN,<NA>,<NA>,-0.73,NaN,NaN


In [42]:
df.head()

,Child ID,Gender,Date of Birth,Location - Sub Level 1,Program Label,Date of Testing,Examiner,Adaptive Sum of Scaled Scores,Adaptive Developmental Quotient,Adaptive Percentile Rank,Adaptive 95% Confidence Interval,Adaptive RDI,Adaptive Z-Score,Adaptive T-Score,Adaptive NCE,Social-Emotional Sum of Scaled Scores,Social-Emotional Developmental Quotient,Social-Emotional Percentile Rank,Social-Emotional 95% Confidence Interval,Social-Emotional RDI,Social-Emotional Z-Score,Social-Emotional T-Score,Social-Emotional NCE,Communication Sum of Scaled Scores,Communication Developmental Quotient,Communication Percentile Rank,Communication 95% Confidence Interval,Communication RDI,Communication Z-Score,Communication T-Score,Communication NCE,Motor Sum of Scaled Scores,Motor Developmental Quotient,Motor Percentile Rank,Motor 95% Confidence Interval,Motor RDI,Motor Z-Score,Motor T-Score,Motor NCE,Cognitive Sum of Scaled Scores,Cognitive Developmental Quotient,Cognitive Percentile Rank,Cognitive 95% Confidence Interval,Cognitive RDI,Cognitive Z-Score,Cognitive T-Score,Cognitive NCE,BDI-3 Total Sum of Scaled Scores,BDI-3 Total Developmental Quotient,BDI-3 Total Percentile Rank,BDI-3 Total 90% Confidence Interval,BDI-3 Total 95% Confidence Interval,BDI-3 Total Z-Score,BDI-3 Total T-Score,BDI-3 Total NCE,Adaptive-Self Care RS,Adaptive-Self Care SS,Adaptive-Self Care PR,Adaptive-Self Care AE,Adaptive-Self Care RDI,Adaptive-Self Care CSS,Adaptive-Self Care CSS 90%,Adaptive-Self Care Z-Score,Adaptive-Self Care T-Score,Adaptive-Self Care NCE,Adaptive-Personal Responsibility RS,Adaptive-Personal Responsibility SS,Adaptive-Personal Responsibility PR,Adaptive-Personal Responsibility AE,Adaptive-Personal Responsibility RDI,Adaptive-Personal Responsibility CSS,Adaptive-Personal Responsibility CSS 90%,Adaptive-Personal Responsibility Z-Score,Adaptive-Personal Responsibility T-Score,Adaptive-Personal Responsibility NCE,Social Emotional-Adult Interaction RS,Social Emotional-Adult Interaction SS,Social Emotional-Adult Interaction PR,Social Emotional-Adult Interaction AE,Social Emotional-Adult Interaction RDI,Social Emotional-Adult Interaction CSS,Social Emotional-Adult Interaction CSS 90%,Social Emotional-Adult Interaction Z-Score,Social Emotional-Adult Interaction T-Score,Social Emotional-Adult Interaction NCE,Social Emotional-Peer Interaction RS,Social Emotional-Peer Interaction SS,Social Emotional-Peer Interaction PR,Social Emotional-Peer Interaction AE,Social Emotional-Peer Interaction RDI,Social Emotional-Peer Interaction CSS,Social Emotional-Peer Interaction CSS 90%,Social Emotional-Peer Interaction Z-Score,Social Emotional-Peer Interaction T-Score,Social Emotional-Peer Interaction NCE,Social Emotional-Self Concept / Social Role RS,Social Emotional-Self Concept / Social Role SS,Social Emotional-Self Concept / Social Role PR,Social Emotional-Self Concept / Social Role AE,Social Emotional-Self Concept / Social Role RDI,Social Emotional-Self Concept / Social Role CSS,Social Emotional-Self Concept / Social Role CSS 90%,Social Emotional-Self Concept / Social Role Z-Score,Social Emotional-Self Concept / Social Role T-Score,Social Emotional-Self Concept / Social Role NCE,Communication-Receptive Communication RS,Communication-Receptive Communication SS,Communication-Receptive Communication PR,Communication-Receptive Communication AE,Communication-Receptive Communication RDI,Communication-Receptive Communication CSS,Communication-Receptive Communication CSS 90%,Communication-Receptive Communication Z-Score,Communication-Receptive Communication T-Score,Communication-Receptive Communication NCE,Communication-Expressive Communication RS,Communication-Expressive Communication SS,Communication-Expressive Communication PR,Communication-Expressive Communication AE,Communication-Expressive Communication RDI,Communication-Expressive Communication CSS,Communication-Expressive Communication CSS 90%,Communication-Expressive Communication Z-Score,Communication-Expressive Comm

In [46]:
age_and_elig = df.iloc[:,-2:]
age_and_elig 

,Age at Testing,Eligibility
0,2.978082,40% delay
1,2.978082,40% delay
2,3.758904,40% delay
3,3.758904,40% delay
4,3.306849,40% delay
...,...,...
23939,2.145205,None
23940,3.304110,None
23941,0.698630,None
23942,2.810959,None


In [49]:
bdi_df = pd.concat([general, bdi, age_and_elig], axis=1)
bdi_df

,Child ID,Gender,Date of Birth,Location - Sub Level 1,Program Label,Date of Testing,Examiner,BDI-3 Total Sum of Scaled Scores,BDI-3 Total Developmental Quotient,BDI-3 Total Percentile Rank,BDI-3 Total 90% Confidence Interval,BDI-3 Total 95% Confidence Interval,BDI-3 Total Z-Score,BDI-3 Total T-Score,BDI-3 Total NCE,Age at Testing,Eligibility
0,404042,M,2020-03-17,Upper Cumberland,BDI-3 Annual Evaluation,2023-03-09,Stacy Holden,15.0,40.0,<0.1,39-43,38-44,0.00,NaN,NaN,2.978082,40% delay
1,404042,M,2020-03-17,Upper Cumberland,BDI-3 Annual Evaluation,2023-03-09,Stacy Holden,15.0,40.0,<0.1,39-43,38-44,0.00,NaN,NaN,2.978082,40% delay
2,475636,F,2019-10-16,Northwest,BDI-3 Annual Evaluation,2023-07-19,Jennifer Hodges,24.0,44.0,<0.1,43-47,42-48,-3.87,13.0,<1,3.758904,40% delay
3,475636,F,2019-10-16,Northwest,BDI-3 Annual Evaluation,2023-07-19,Jennifer Hodges,24.0,44.0,<0.1,43-47,42-48,-3.87,13.0,<1,3.758904,40% delay
4,476193,M,2019-11-08,Memphis Delta,BDI-3 Annual Evaluation,2023-02-27,Kayla Coley,37.0,51.0,<0.1,49-53,48-54,0.00,17.0,<1,3.306849,40% delay
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23939,544499,M,2020-11-25,Greater Nashville,BDI-3 Eligibility Evaluation,2023-01-17,Tracey Hardiman,NaN,NaN,NaN,<NA>,<NA>,0.00,NaN,NaN,2.145205,None
23940,549646,M,2020-05-02,Southeast Tennessee,BDI-3 Eligibility Evaluation,2023-08-21,Mollee Favre,NaN,NaN,NaN,<NA>,<NA>,-3.60,NaN,NaN,3.304110,None
23941,552599,F,2022-08-27,First Tennessee,BDI-3 Eligibility Evaluation,2023-05-09,Karen McMillion,NaN,NaN,NaN,<NA>,<NA>,-0.40,NaN,NaN,0.698630,None
23942,SP00659,M,2020-07-30,East Tennessee,BDI-3 Eligibility Evaluation,2023-05-22,Kelly Brackney,NaN,NaN,NaN,<NA>,<NA>,-0.73,NaN,NaN,2.810959,None
